In [1]:
import os
import glob
import shutil
import random

# Define your original dataset directory and the new structure
original_dataset_dir = '../.local/datasets/IoT_ObjectDetection'  # Path to your original dataset folder
new_dataset_dir = '../.local/datasets/IoT_ObjectDetection_Yolo'  # Path to the new structured dataset

In [2]:
# Create new directories for train and validation
os.makedirs(os.path.join(new_dataset_dir, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, 'train', 'labels'), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, 'val', 'labels'), exist_ok=True)

# Collect all class folders
class_folders = [d for d in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, d))]
class_folders.sort()  # Sort class folders to maintain consistent class index

# Create a mapping from class name to class index
class_mapping = {class_name: idx for idx, class_name in enumerate(class_folders)}

for class_name in class_folders:
    class_folder = os.path.join(original_dataset_dir, class_name)
    image_files = glob.glob(os.path.join(class_folder, '*.jpg'))

    # Randomly sample 10 images for validation
    val_samples = random.sample(image_files, min(10, len(image_files)))

    for img_path in image_files:
        # Get the base name without extension
        base_name = os.path.splitext(os.path.basename(img_path))[0]

        # Move the image to the train images folder if it's not in the validation sample
        if img_path not in val_samples:
            new_img_path = os.path.join(new_dataset_dir, 'train', 'images', os.path.basename(img_path))
            shutil.copy(img_path, new_img_path)

            # Move the corresponding label file
            label_file = os.path.join(class_folder, f"{base_name}.txt")
            if os.path.exists(label_file):
                new_label_path = os.path.join(new_dataset_dir, 'train', 'labels', f"{base_name}.txt")
                with open(label_file, 'r') as f:
                    with open(new_label_path, 'w') as f_out:
                        # Adjust the class index in the label file
                        for line in f.readlines():
                            parts = line.strip().split()
                            if len(parts) > 0:
                                # Replace the original class index with the correct index from the mapping
                                original_class_index = parts[0]
                                new_class_index = class_mapping[class_name]
                                # Write in YOLO format (new_class_index x_center y_center width height)
                                f_out.write(f"{new_class_index} {' '.join(parts[1:])}\n")

        else:
            # Move the selected validation image
            new_img_path = os.path.join(new_dataset_dir, 'val', 'images', os.path.basename(img_path))
            shutil.copy(img_path, new_img_path)

            # Move the corresponding validation label file
            label_file = os.path.join(class_folder, f"{base_name}.txt")
            if os.path.exists(label_file):
                new_label_path = os.path.join(new_dataset_dir, 'val', 'labels', f"{base_name}.txt")
                with open(label_file, 'r') as f:
                    with open(new_label_path, 'w') as f_out:
                        # Adjust the class index in the label file
                        for line in f.readlines():
                            parts = line.strip().split()
                            if len(parts) > 0:
                                # Replace the original class index with the correct index from the mapping
                                original_class_index = parts[0]
                                new_class_index = class_mapping[class_name]
                                # Write in YOLO format (new_class_index x_center y_center width height)
                                f_out.write(f"{new_class_index} {' '.join(parts[1:])}\n")

print("Dataset reorganized with random samples for validation and class indices adjusted!")

Dataset reorganized with random samples for validation and class indices adjusted!


In [3]:
import yaml

# Get class names from the folders in the original dataset directory
classes = [d for d in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, d))]
classes.sort()

data_dict = {
    'train': "../IoT_ObjectDetection_Yolo/train",
    'val': "../IoT_ObjectDetection_Yolo/val",
    'nc': len(classes),
    'names': classes,
}

dataset_yaml_path = '/home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/.local/datasets/IoT_ObjectDetection/data.yaml'
# Save the YAML file
with open(dataset_yaml_path, 'w') as f:
    yaml.dump(data_dict, f)

print("data.yaml file created!")

data.yaml file created!


In [4]:
from ultralytics import YOLO

TRAINED_MODEL_PATH = '../.local/models/object_detection/yolov11n_trained.pt'
# Load the YOLOv8 model
model = YOLO('yolo11n.pt')

# Train the model
model.train(data=dataset_yaml_path, epochs=50, imgsz=640, batch=16)  # Adjust parameters as needed

print("Training completed!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/tobias/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 25.4MB/s]


Ultralytics 8.3.28 🚀 Python-3.10.15 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 7789MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/.local/datasets/IoT_ObjectDetection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

train: Scanning /home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/.local/datasets/IoT_ObjectDetection_Yolo/train/labels... 2098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2098/2098 [00:01<00:00, 1309.42it/s]


train: New cache created: /home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/.local/datasets/IoT_ObjectDetection_Yolo/train/labels.cache


...

      50/50       2.4G     0.3899     0.2008     0.8513          3        640: 100%|██████████| 132/132 [00:35<00:00,  3.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.31it/s]

                   all        158        215      0.992      0.991      0.995      0.955



50 epochs completed in 0.567 hours.
Optimizer stripped from /home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from /home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/runs/detect/train/weights/best.pt, 5.5MB

Validating /home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/runs/detect/train/weights/best.pt...
Ultralytics 8.3.28 🚀 Python-3.10.15 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 7789MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  2.30it/s]


                   all        158        215      0.992      0.991      0.995      0.955
          HueLampGreen         19         19      0.994          1      0.995      0.964
         HueLampPurple         19         19      0.994          1      0.995      0.985
            HueLampRed         20         20      0.994          1      0.995      0.944
 HueLampWhiteHalfRound         20         20      0.993          1      0.995      0.986
         HueLampYellow         20         20      0.994          1      0.995      0.945
           OfficeLight         20         30      0.973      0.967      0.994      0.984
          OfficeWindow         20         53      0.998      0.962      0.994      0.956
            Tractorbot         20         34      0.999          1      0.995      0.879
Speed: 0.8ms preprocess, 1.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /home/tobias/Desktop/Uni/IMP/IMP_Magical_Gaze-based_Device_Control/runs/detect/train
Training comple

In [5]:
# Save the trained model
model.save(TRAINED_MODEL_PATH)

In [ ]:
from ultralytics import YOLO

TRAINED_MODEL_PATH = '../.local/models/object_detection/yolov8s_trained.pt'

model = YOLO(TRAINED_MODEL_PATH)
# Perform validation
results = model.val(data=dataset_yaml_path, save_json=True)  # Validate and save results in JSON format

In [ ]:
import os
import glob
import yaml
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the YOLOv8 model
model = YOLO(TRAINED_MODEL_PATH)

dataset_dir = "/home/tobias/Desktop/Uni/IMP/datasets/"

# predict on the validation set and visualize the results
with open('data.yaml') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
print(data['val'])
val_images = glob.glob(os.path.join(dataset_dir, data['val'], 'images', '*.jpg'))

for img_path in val_images:
    results = model(img_path)
    # Process results list
    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        obb = result.obb  # Oriented boxes object for OBB outputs
        result.show()  # display to screen
    